In [ ]:
# hide
# default_exp L1_features_gen
# from nbdev.showdoc import *

# 01 Features generation

> scripts for generating features from raw. I will be reading from the **raw** (zip file) and output all the features (both on user and geolocation level) to **Data/L1**

# Library

In [ ]:
#exports
import pandas as pd
import numpy as np
import os
from zipfile import ZipFile
from scipy import spatial
import matplotlib.pyplot as plt

from tsfresh import extract_features
from tsfresh.feature_selection.relevance import calculate_relevance_table
import tsfresh

# Functions

In [ ]:
#exports
def read_tsv(file:str)->pd.DataFrame:
    return pd.read_csv(file,  compression='gzip', sep='\t')

def gzip_reading(gzip_file)->dict:
    'Read all tsv.gz files in the zip file and returning a dictionary (key:filename, value:data)'
    archive = ZipFile(gzip_file, 'r')
    files = {name: archive.open(name) for name in archive.namelist() if
     (name.endswith('.gz') and not name.startswith('_'))}
    files_names = [i.split('.')[0] for i in files.keys()]
    
    # reading the designated files into dict
    dt = {}
    for name, key in zip(files_names, files.keys()):
        dt[name] = read_tsv(files[key])
    return dt

def school_plan__features(data:pd.DataFrame)->pd.DataFrame:
    'Calculate the number of school (and its associated types) within the planning area'
    school_count = data.groupby('planning_area', as_index=False).size().rename({'size':'number_school'}, axis=1)
    school_cat_count = (data
                        .groupby(['planning_area','category'], as_index=False).size()
                        .rename({'size':'number_school'}, axis=1)
                        .pivot(index='planning_area', columns='category', values='number_school').fillna(0)
                       ).reset_index()
    return school_count.merge(school_cat_count)

def translate_latlng(input:list)->list:
    'Translating the lat lng into tuple format, to be used to mathematically identify the nearest neighbor'
    latlong_location_str = [i.replace(" ","").replace("\"", "").split(",") for i in input]
    latlong_location_num = [(float(x), float(y)) for x, y in latlong_location_str]
    return latlong_location_num

def kdtree_neighbors(reference:list, query_data:list)->list:
    'Identify the nearest neighbor for *query_data*[list of (lat,lng)] to the *reference*[list of (lat,lng)], returning the matching reference index'
    tree = spatial.KDTree(reference)
    return tree.query(query_data)[1]

def train_plan__latlng(data:pd.DataFrame)->pd.DataFrame:
    list_tuple_latlng =  translate_latlng(data['latlong'])
    data['lat'] = [i[0] for i in list_tuple_latlng]
    data['lng'] = [i[1] for i in list_tuple_latlng]
    return data.groupby('planning_area', as_index=False)[['lat','lng']].median()

def train_plan__nbusers(data:pd.DataFrame)->pd.DataFrame:
    return data.groupby('planning_area', as_index=False).size().rename({'size':'users_nb'}, axis=1)

def train_time_features(data:pd.DataFrame):
    'Modify the train dataset inplace to generate time features (*month_delta* and *account start year*)'
    data['account_start_date'] = pd.to_datetime(data['account_start_date'])
    data['reference_date'] = pd.to_datetime(data['reference_date'])
    data['month_delta'] = [int(i/np.timedelta64(1, 'M')) for i in (data['reference_date'] - data['account_start_date'])]
    data['account_start_year'] = [i.year for i in data['account_start_date']]
    
def census_income_median(data:pd.DataFrame)->pd.DataFrame:    
    'Calculate the median income and working pop for region excluding outliers (above 10K SGD and 0 SGD)'
    return (data
            .loc[~data['variable'].isin([12000,0])]
            .sort_values('value', ascending=False)
            .drop_duplicates(['planning_area'])
            .rename({'variable':'med_income','value':'working_pop'}, axis=1)
            .drop('working_pop', axis=1)
           )

def census_income_avg(data:pd.DataFrame)->pd.DataFrame:
    'Calculate average income based on the avg(pop * income)'
    return (census_income_dt_melt
            .assign(temp=census_income_dt_melt.eval('variable * value'))
             .groupby('planning_area')[['temp','value']]
             .sum()
             .eval('temp / value')
             .to_frame('avg_income')
             .reset_index()
            )

def gini_processing(census_perc_dt_melt:pd.DataFrame)->pd.DataFrame:
    'Parsing the income percent into np.array for gini calculation'
    income_per_dict = {'no_working_person_percent':0, 
     'below_sgd_1000_percent':.05,
     'sgd_10000_over_percent':1, 
     'sgd_1000_to_1999_percent':.15, 
     'sgd_2000_to_2999_percent':.25,
     'sgd_3000_to_3999_percent':.35, 
     'sgd_4000_to_4999_percent':.45, 
     'sgd_5000_to_5999_percent':.55,
     'sgd_6000_to_6999_percent':.65, 
     'sgd_7000_to_7999_percent':.75, 
     'sgd_8000_to_8999_percent':.85,
     'sgd_9000_to_9999_percent':.95}

    census_perc_dt_melt['variable'] = census_perc_dt_melt['variable'].map(income_per_dict)
    census_perc_dt_melt.sort_values(['planning_area','variable'], inplace=True)
    census_perc_dt_melt = census_perc_dt_melt.assign(value = census_perc_dt_melt['value'].mul(10).astype(int))

    unique_areas = census_perc_dt_melt['planning_area'].unique()
    gini_dict={}
    for i in unique_areas: # looking at individual planning area seperately
        tmp_dt = census_perc_dt_melt.loc[census_perc_dt_melt['planning_area']==i][['variable','value']] 
        tmp_list = [np.array([x]*y) for x, y in zip(tmp_dt['variable'], tmp_dt['value'])] # creation y records based on x value
        out = np.hstack(tmp_list).squeeze()
        gini_dict[i] = gini(out)

    gini_dt = pd.Series(gini_dict).to_frame().reset_index()
    gini_dt.columns = ['planning_area','gini_coef']
    return gini_dt

# adapted from https://github.com/oliviaguest/gini
def gini(array):
    'Calculate the Gini coefficient of a numpy array.'
    # based on bottom eq: http://www.statsdirect.com/help/content/image/stat0206_wmf.gif
    # from: http://www.statsdirect.com/help/default.htm#nonparametric_methods/gini.htm
    array = array.flatten() #all values are treated equally, arrays must be 1d
    if np.amin(array) < 0:
        array -= np.amin(array) #values cannot be negative
    array += 0.0000001 #values cannot be 0
    array = np.sort(array) #values must be sorted
    index = np.arange(1,array.shape[0]+1) #index per array element
    n = array.shape[0]#number of array elements
    return ((np.sum((2 * index - n  - 1) * array)) / (n * np.sum(array))) #Gini coefficient


from transformers import pipeline
# Allocate a pipeline for sentiment-analysis
classifier = pipeline('sentiment-analysis')
def census_sentiment_analy(input:str)->tuple:
    output = classifier(input)
    return output

def convert_sent2score(input:dict):
    'Extract and convert the sentimental assignemnt to 0 (Negative), 0.5 (Neutral), 1 (Positive)'
    score_list=[]
    for i in input:
        if i['score'] > 0 and i['score'] < 0.7:
            score_list.append(0.5)
        elif i['label'] == 'POSITIVE':
            score_list.append(1.0)
        elif i['label'] == 'NEGATIVE':
            score_list.append(0.0)
    return score_list

def transform_churn_series(extracted_features_dt:pd.DataFrame, dt_churn:pd.DataFrame)->pd.DataFrame:
    'Extract churn data for significant test in x'
    churn_dt = dt_churn[['msisdn','churn']]

    churn_dt_ordered = (extracted_features_dt.reset_index()[['index']]
     .rename({'index':'msisdn'}, axis=1)
     .merge(churn_dt)
#      .drop(['voice_incoming__maximum'], axis=1)
     .set_index('msisdn')
    )

    return churn_dt_ordered

In [ ]:
# hide
dt = gzip_reading(os.path.join('Data','L0','telco_demo_datasets.zip'))

# schools
Allocating the schools data into planning area (base on nearest neighbors), to aggregate it to planning area

In [ ]:
# list of schools latlng to be used for query
latlong_school =  translate_latlng(dt['telco_schools']['latLong'])

# generating unique lat long - planning area data based on users that is used for references mapping
dt_location = dt['telco_train'].groupby(['latlong','planning_area'], as_index=False)['msisdn'].nunique()
latlong_users =  translate_latlng(dt_location['latlong'])

# ktree to find nearest users's planning area on telco train data 
query_idx = kdtree_neighbors(reference=latlong_users, query_data=latlong_school)
dt['telco_schools']['planning_area'] = [dt_location['planning_area'][i] for i in query_idx]
dt['telco_schools']

,category,name,latLong,planning_area
0,Primary Schools,Alexandra Primary,"""1.291298368, 103.8239405""",BUKIT MERAH
1,Primary Schools,Gan Eng Seng Primary,"""1.2855944990000001, 103.8155471""",BUKIT MERAH
2,Primary Schools,Zhangde Primary,"""1.284279434, 103.8261758""",BUKIT MERAH
3,Secondary Schools,Crescent Girls,"""1.293318804, 103.8175441""",QUEENSTOWN
4,Secondary Schools,Gan Eng Seng,"""1.289070478, 103.8237333""",BUKIT MERAH
...,...,...,...,...
368,Primary Schools,Pioneer Primary,"""1.348720261, 103.6948675""",JURONG WEST
369,Primary Schools,Qihua Primary,"""1.4420353, 103.7883394""",WOODLANDS
370,Secondary Schools,Riverside Secondary,"""1.441077283, 103.7883131""",WOODLANDS
371,Secondary Schools,Westwood Secondary,"""1.353736319, 103.7017718""",JURONG WEST


In [ ]:
planning_area_school = school_plan__features(dt['telco_schools'])
planning_area_school.columns = [i.lower().replace(" ","_") for i in planning_area_school.columns]
planning_area_school.head()

,planning_area,number_school,integrated_schools,primary_schools,secondary_schools
0,ANG MO KIO,17,1.0,8.0,8.0
1,BEDOK,24,1.0,12.0,11.0
2,BISHAN,15,3.0,4.0,8.0
3,BUKIT BATOK,11,0.0,6.0,5.0
4,BUKIT MERAH,13,0.0,8.0,5.0


In [ ]:
planning_area_school.to_pickle((os.path.join("Data","L1", "geo_school.pkl")))

# train
reference_date - 2020-04-15

To acquire: 
- mean/median lat and lng
- number of users

For each planning area

In [ ]:
train_dt = dt['telco_train'].copy()
plan_area_latlng = train_plan__latlng(train_dt) # extracting numerical (lat,lng) from data
plan_area_users_nb = train_plan__nbusers(train_dt)
plan_area_dt = plan_area_latlng.merge(plan_area_users_nb)
plan_area_dt.head()

,planning_area,lat,lng,users_nb
0,ANG MO KIO,1.371236,103.847778,1012
1,BEDOK,1.331222,103.928134,1479
2,BISHAN,1.355431,103.839107,127
3,BUKIT BATOK,1.351252,103.750406,1276
4,BUKIT MERAH,1.279427,103.822536,429


In [ ]:
plan_area_dt.shape

(30, 4)

In [ ]:
plan_area_dt.to_pickle((os.path.join("Data","L1", "geo_coor.pkl")))

## user train 
To prepare for modeling later on, categories fields are modified.
- **churn** - 1(churned), 0(no)
- **contract** - yearly :1, monthly:0
- **internet_service** - fiber :1, no:0

some time based features (based on contracts- when does it starts, how long has it been going) are also generated
- **account_start_year** - how long are these users with the telco
- **month_delta** - how many months are they from the contract start (ref date - contract date)

In [ ]:
# creating time features based on contracts
train_time_features(train_dt)

train_dt = train_dt[['msisdn', 'churn', 'age', 'contract',
       'internet_service', 'account_start_year','planning_area', 'month_delta']]
# dropping some columns that would not make a lot of sense in this case
# 'address','name', 'birthday', 'account_start_date', 'reference_date','gender'

train_dt.loc[:,'churn'] = train_dt['churn'].astype(int)
contract_dict = {'monthly':0, 'two-year':1}
train_dt.loc[:,'contract'] = train_dt['contract'].map(contract_dict)
internet_service = {'fiber':1, 'no':0}
train_dt.loc[:,'internet_service'] = train_dt['internet_service'].map(internet_service)

In [ ]:
train_dt.head()

,msisdn,churn,age,contract,internet_service,account_start_year,planning_area,month_delta
0,6048764759382,0,44,0,1,2018,TOA PAYOH,25
1,1948924115781,0,21,1,1,2016,GEYLANG,45
2,5938778408016,0,57,0,1,2018,JURONG WEST,16
3,975351393328,0,49,1,0,2014,BEDOK,72
4,1587148418583,0,24,0,0,2011,BEDOK,101


In [ ]:
train_dt.to_pickle((os.path.join("Data","L1", "user_train.pkl")))

## geo train
Converting the train data to geography level,since some trends/features would be better measured at geo level.

In [ ]:
geo_train = train_dt.groupby('planning_area')[['age','contract','internet_service','account_start_year','month_delta','churn']].mean().reset_index()
geo_train.head()

,planning_area,age,contract,internet_service,account_start_year,month_delta,churn
0,ANG MO KIO,38.967391,0.531621,0.443676,2014.265810,65.870553,0.093874
1,BEDOK,39.557133,0.555105,0.465855,2014.260987,65.876944,0.077755
2,BISHAN,38.417323,0.511811,0.527559,2014.070866,67.992126,0.196850
3,BUKIT BATOK,40.536050,0.530564,0.472571,2014.325235,65.303292,0.063480
4,BUKIT MERAH,39.228438,0.543124,0.510490,2014.235431,66.386946,0.118881


In [ ]:
geo_train.to_pickle(os.path.join("Data","L1", "geo_train.pkl"))

# locations

printing sample data of locations (top 10 visiits location for users)

In [ ]:
cur_dt = dt['telco_locations']
cur_dt.head()

,latitude,longitude,msisdn
0,1.326087,103.898460,6048764759382
1,1.301823,103.904991,1948924115781
2,1.301894,103.904761,5938778408016
3,1.334966,103.745927,975351393328
4,1.291109,103.812621,1587148418583


assigne the lat long to a planning area based on the avilable user data

In [ ]:
# list of schools latlng to be used for query
latlong_loc =  [(float(x), float(y)) for x,y in zip(cur_dt['latitude'],cur_dt['longitude'])]

# generating unique lat long - planning area data based on users that is used for references mapping
dt_location = dt['telco_train'].groupby(['latlong','planning_area'], as_index=False)['msisdn'].nunique()
latlong_users =  translate_latlng(dt_location['latlong'])

# ktree to find nearest users's planning area on telco train data 
query_idx = kdtree_neighbors(reference=latlong_users, query_data=latlong_loc)
dt['telco_locations']['planning_area'] = [dt_location['planning_area'][i] for i in query_idx]

### geo location

In [ ]:
dt['telco_locations']

,latitude,longitude,msisdn,planning_area
0,1.326087,103.898460,6048764759382,GEYLANG
1,1.301823,103.904991,1948924115781,MARINE PARADE
2,1.301894,103.904761,5938778408016,MARINE PARADE
3,1.334966,103.745927,975351393328,JURONG EAST
4,1.291109,103.812621,1587148418583,QUEENSTOWN
...,...,...,...,...
29995,1.300675,103.838027,197681231203,BUKIT MERAH
29996,1.301725,103.904361,6183326134523,MARINE PARADE
29997,1.356236,103.990446,9770826000249,TAMPINES
29998,1.309874,103.901078,5855470225901,GEYLANG


In [ ]:
dt['telco_locations'].to_pickle(os.path.join("Data","L1", "geo_location.pkl"))

### users location

In [ ]:
user_loc = dt['telco_locations'].copy()
user_loc.sort_values('planning_area')
user_loc['value']=1
user_loc = user_loc[['msisdn','planning_area','value']].pivot_table(index=['msisdn'], columns = 'planning_area').reset_index().fillna(0)

In [ ]:
# flatten columns
tmp_columns = [x if y=='' else y.lower() for x,y in zip(user_loc.columns.get_level_values(0), user_loc.columns.get_level_values(1))]
# applying manipulated columns
user_loc.columns = tmp_columns

In [ ]:
user_loc.head()

,msisdn,ang mo kio,bedok,bishan,bukit batok,bukit merah,bukit timah,changi,clementi,geylang,...,marine parade,novena,outram,pasir ris,queenstown,rochor,serangoon,tampines,tanglin,toa payoh
0,351144701,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1041740389,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1529121439,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2189894234,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,2504021639,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


this might be too spare for usage with users data and may require additional processing...

In [ ]:
user_loc.to_pickle(os.path.join("Data","L1", "user_location.pkl"))

# census
condensing the features into
- **med_income**
- **avg_income**
- **gini_coef**
- **pop**
- **working_pop**

In [ ]:
census_dt = dt['telco_census']
census_dt.head()

,planning_area,total,below_sgd_1000,no_working_person,sgd_10000_over,sgd_1000_to_1999,sgd_2000_to_2999,sgd_3000_to_3999,sgd_4000_to_4999,sgd_5000_to_5999,...,sgd_10000_over_percent,sgd_1000_to_1999_percent,sgd_2000_to_2999_percent,sgd_3000_to_3999_percent,sgd_4000_to_4999_percent,sgd_5000_to_5999_percent,sgd_6000_to_6999_percent,sgd_7000_to_7999_percent,sgd_8000_to_8999_percent,sgd_9000_to_9999_percent
0,ANG MO KIO,59705,3136,8467,12137,5549,6055,5236,4565,3959,...,20.328281,9.294029,10.141529,8.769785,7.645926,6.630935,5.622645,4.875639,4.090110,3.167239
1,BEDOK,91224,3782,11143,25281,6857,7476,7362,6596,5825,...,27.713102,7.516662,8.195212,8.070245,7.230553,6.385381,5.967728,4.770674,4.173244,3.615277
2,BISHAN,27457,595,2855,10264,1166,1390,1717,1616,1594,...,37.382088,4.246640,5.062461,6.253414,5.885567,5.805441,6.133226,5.907419,5.026041,5.728958
3,BUKIT BATOK,44133,1136,3452,13088,2849,3486,3723,3537,3264,...,29.655813,6.455487,7.898851,8.435864,8.014411,7.395826,6.852015,5.619378,4.921487,4.355018
4,BUKIT MERAH,55627,4142,10311,10685,5324,4495,3767,3679,3602,...,19.208298,9.570892,8.080608,6.771891,6.613695,6.475273,5.245654,4.589498,4.210186,3.252018


In [ ]:
# gini calculation
## columns control
index_col = ['planning_area']
perc_columns = [i for i in census_dt.columns if i in index_col or 'percent' in i]
## subset of dt
census_perc_dt = census_dt[perc_columns]
## gini
census_perc_dt_melt = census_perc_dt.melt(id_vars='planning_area')
census_gini_dt = gini_processing(census_perc_dt_melt)

# income avg and med calculation
## subset of dt
census_income_dt = census_dt[['planning_area', 'below_sgd_1000', 'no_working_person',
       'sgd_10000_over', 'sgd_1000_to_1999', 'sgd_2000_to_2999',
       'sgd_3000_to_3999', 'sgd_4000_to_4999', 'sgd_5000_to_5999',
       'sgd_6000_to_6999', 'sgd_7000_to_7999', 'sgd_8000_to_8999',
       'sgd_9000_to_9999']]
census_income_dt_melt = census_income_dt.melt(id_vars='planning_area')
income_dict = {'no_working_person':0, 
 'below_sgd_1000':500,
 'sgd_10000_over':12000, 
 'sgd_1000_to_1999':1500, 
 'sgd_2000_to_2999':2500,
 'sgd_3000_to_3999':3500, 
 'sgd_4000_to_4999':4500, 
 'sgd_5000_to_5999':5500,
 'sgd_6000_to_6999':6500, 
 'sgd_7000_to_7999':7500, 
 'sgd_8000_to_8999':8500,
 'sgd_9000_to_9999':9500}
census_income_dt_melt['variable'] = census_income_dt_melt['variable'].map(income_dict)
census_income_median_dt = census_income_median(census_income_dt_melt)
census_income_average_dt = census_income_avg(census_income_dt_melt)

# raw data
census_total_dt = census_dt[['planning_area','total','no_working_person']]
# combining all the calculated income features
census_concat_dt = (census_income_median_dt
                    .merge(census_income_average_dt)
                    .merge(census_gini_dt)
                    .merge(census_total_dt)
                   ).rename({'total':'pop'}, axis=1)
census_concat_dt['working_pop'] = census_concat_dt['pop'] - census_concat_dt['no_working_person']
census_concat_dt.drop('no_working_person', axis=1, inplace=True)

census_concat_dt.head()

,planning_area,med_income,avg_income,gini_coef,pop,working_pop
0,JURONG WEST,3500,6000.685168,0.336021,78083,72683
1,BEDOK,2500,6066.907469,0.383530,91224,80081
2,WOODLANDS,4500,5892.433984,0.336238,68279,64030
3,TAMPINES,3500,6456.678897,0.323235,73591,68297
4,ANG MO KIO,2500,5254.007202,0.427458,59705,51238


In [ ]:
census_concat_dt.to_pickle((os.path.join("Data","L1", "geo_census.pkl")))

# reviews
Overtime data (monthly per user) for 6 months.
- used hugging face (pretrained bert) to run a sentimental analysis
- assigned positive to 1, neutral to 0.5, negative to 0
- generate time series features via tsfresh
- significant test with churns.

This takes significant amount of time to run on CPU (as well as lots of dependency), seeing that there is no significant trends identified with the current pipeline, it should perhaps be commented out on normal runs

In [ ]:
reviews_dt = dt['telco_reviews']
reviews_dt.head()

,msisdn,date,feedback
0,6048764759382,2019-10-31,Detailed bluetooth weighted component biotechn...
1,1948924115781,2019-10-31,Functionality plenty clients magical baghdad s...
2,5938778408016,2019-10-31,Imports rj mardi henry mm ones optical laptops...
3,975351393328,2019-10-31,Stephen eminem valued evidence prescription si...
4,1587148418583,2019-10-31,Cap designated choose entertainment discussion...


In [ ]:
# feeding 1000 reviews at a time into transfomer sentimental analysis from hugging face
n=0
n_top=0
sent_list=[]
while n_top<len(reviews_dt):
    n_top+=1000
    sent_list.append(census_sentiment_analy(reviews_dt['feedback'].tolist()[n:n_top]))
    n+=1000

# flattening the list
flat_list = [item for sublist in sent_list for item in sublist]
reviews_dt['sentimental_score'] = convert_sent2score(flat_list)

In [ ]:
reviews_dt.head()

,msisdn,date,feedback,sentimental_score
0,6048764759382,2019-10-31,Detailed bluetooth weighted component biotechn...,1.0
1,1948924115781,2019-10-31,Functionality plenty clients magical baghdad s...,1.0
2,5938778408016,2019-10-31,Imports rj mardi henry mm ones optical laptops...,0.0
3,975351393328,2019-10-31,Stephen eminem valued evidence prescription si...,0.5
4,1587148418583,2019-10-31,Cap designated choose entertainment discussion...,0.0


In [ ]:
reviews_sub_dt = reviews_dt[['msisdn','date','sentimental_score']]

In [ ]:
reviews_sub_dt.to_pickle((os.path.join("Data","L1", "user_senti_score_OT.pkl")))

In [ ]:
# features to calculate
features_dict = {"maximum": None, "minimum": None, "mean": None, "absolute_sum_of_changes":None,
                 "benford_correlation":None, "mean_change":None, 'sum_values':None, 'linear_trend': [{'attr': 'slope'}, {'attr': 'intercept'}], 
                }
kind_to_fc_parameters = {
    "sentimental_score": features_dict
}
# calc all the ts features
extracted_features_dt = extract_features(reviews_sub_dt, column_id = "msisdn", column_sort = "date", kind_to_fc_parameters = kind_to_fc_parameters)

# dropping na records
extracted_features_dt.dropna(inplace=True)

# generating index for churn for significant check
churn_dt = transform_churn_series(extracted_features_dt = extracted_features_dt, dt_churn = dt['telco_train'])

# significant test and retain only relevent features
relevance_dt = calculate_relevance_table(extracted_features_dt, churn_dt['churn'], ml_task='auto', n_jobs=4)
sig_columns = relevance_dt.loc[relevance_dt['relevant']==True, 'feature'].tolist()
extracted_sig_features_dt = extracted_features_dt[sig_columns]

Feature Extraction: 100%|██████████████████████████████████████████████████████████████| 30/30 [00:11<00:00,  2.65it/s]


In [ ]:
# settings
tsfresh.feature_extraction.settings.from_columns(extracted_features_dt[sig_columns])

{}

In [ ]:
print(f"number of sig features: {len(extracted_sig_features_dt.columns)}")

number of sig features: 0


sadly, it seems the sentimental scores time series trends has little to no impact to the churns (None of the features manage to pass the significant testing in tsfresh).....

# cdr
Overtime CDR data (monthly per user) for 6 months.
- generate time series features via tsfresh
- significant test with churns.

In [ ]:
cdr_dt = dt['telco_cdr']
# printing cdr of a single user
cdr_dt.loc[cdr_dt['msisdn']==351144701].head()

,msisdn,date,voice_incoming,voice_outgoing,sms_incoming,sms_outgoing,data_upload,data_download
407,351144701,2019-10-31,52,28,79,11,1176,2374
15407,351144701,2019-11-30,59,34,68,12,1276,2522
30407,351144701,2019-12-31,49,30,60,8,1218,3186
45407,351144701,2020-01-31,64,33,57,7,1283,2517
60407,351144701,2020-02-29,83,33,59,5,1047,3171


In [ ]:
# TS features to calculate
features_dict = {"maximum": None, "minimum": None, "mean": None, "absolute_sum_of_changes":None,
                 "benford_correlation":None, "mean_change":None, 'longest_strike_above_mean':None,
                 'longest_strike_below_mean':None, 'sum_values':None, 'linear_trend': [{'attr': 'slope'}, {'attr': 'intercept'}], 
                }
kind_to_fc_parameters = {
    "voice_incoming": features_dict,
    "voice_outgoing": features_dict, 
    "sms_incoming": features_dict, 
    "sms_outgoing": features_dict,
    "data_upload": features_dict,
    "data_download": features_dict, 
}
# calc all the ts features
extracted_features_dt = extract_features(cdr_dt, column_id="msisdn", column_sort="date", kind_to_fc_parameters = kind_to_fc_parameters)

# dropping na records
extracted_features_dt.dropna(inplace=True)

# generating index for churn for significant check
churn_dt = transform_churn_series(extracted_features_dt= extracted_features_dt, dt_churn = dt['telco_train'])

# significant test and retain only relevent features
relevance_dt = calculate_relevance_table(extracted_features_dt, churn_dt['churn'], ml_task='auto', n_jobs=4)
sig_columns = relevance_dt.loc[relevance_dt['relevant']==True, 'feature'].tolist()
extracted_sig_features_dt = extracted_features_dt[sig_columns]

# settings
tsfresh.feature_extraction.settings.from_columns(extracted_features_dt[sig_columns])

Feature Extraction: 100%|██████████████████████████████████████████████████████████████| 30/30 [00:27<00:00,  1.09it/s]


{'data_download': {'mean_change': None,
  'linear_trend': [{'attr': 'slope'}],
  'absolute_sum_of_changes': None,
  'maximum': None,
  'sum_values': None,
  'mean': None,
  'minimum': None},
 'voice_outgoing': {'mean_change': None, 'linear_trend': [{'attr': 'slope'}]}}

In [ ]:
extracted_sig_features_dt = extracted_sig_features_dt.reset_index().rename({'index':'msisdn'}, axis=1)
extracted_sig_features_dt.head()

,msisdn,data_download__mean_change,"data_download__linear_trend__attr_""slope""",voice_outgoing__mean_change,"voice_outgoing__linear_trend__attr_""slope""",data_download__absolute_sum_of_changes,data_download__maximum,data_download__sum_values,data_download__mean,data_download__minimum
0,351144701,124.0,125.085714,1.0,0.714286,2312.0,3186.0,16764.0,2794.000000,2374.0
1,1041740389,477.2,447.028571,-0.2,-0.542857,2386.0,3977.0,17152.0,2858.666667,1591.0
2,1529121439,157.8,182.485714,0.0,-0.571429,2219.0,3467.0,16909.0,2818.166667,2307.0
3,2189894234,2.2,23.628571,0.8,0.571429,483.0,2805.0,16323.0,2720.500000,2558.0
4,2504021639,286.0,315.200000,0.2,0.000000,1496.0,3636.0,18506.0,3084.333333,2206.0


mostly Time series feature that are associated with 
- **data_download**
- **voice_outgoing**

In [ ]:
extracted_sig_features_dt.to_pickle((os.path.join("Data","L1", "user_cdr_TS_feature.pkl")))

# web
Overtime web data (monthly visits per user) for 6 months.
- generate time series features via tsfresh
- significant test with churns.

In [ ]:
web_dt = dt['telco_web']
web_dt.head()

,msisdn,date,website,visits
0,6048764759382,2019-10-31,www.singtel.com,620
1,6048764759382,2019-10-31,www.starhub.com,267
2,6048764759382,2019-10-31,www.facebook.com,4260
3,6048764759382,2019-10-31,www.instagram.com,78
4,6048764759382,2019-10-31,www.cnn.com,782


In [ ]:
pivot_web = web_dt.pivot(index=['msisdn','date'], columns='website').reset_index()

# flatten columns
tmp_columns = [x if y=='' else y for x,y in zip(pivot_web.columns.get_level_values(0), pivot_web.columns.get_level_values(1))]
tmp_columns_1 = [i.replace("www.","").replace(".com","") for i in tmp_columns]

# applying manipulated columns
pivot_web.columns = tmp_columns_1

In [ ]:
# features to calculate
features_dict = {"maximum": None, "minimum": None, "mean": None, "absolute_sum_of_changes":None,
                 "benford_correlation":None, "mean_change":None, 'sum_values':None, 'linear_trend': [{'attr': 'slope'}, {'attr': 'intercept'}], 
                }
kind_to_fc_parameters = {
    "cnn": features_dict,
    "facebook": features_dict, 
    "instagram": features_dict, 
    "singtel": features_dict,
    "starhub": features_dict
}
# calc all the ts features
extracted_features_dt = extract_features(pivot_web, column_id = "msisdn", column_sort = "date", kind_to_fc_parameters = kind_to_fc_parameters)

# dropping na records
extracted_features_dt.dropna(inplace=True)

# generating index for churn for significant check
churn_dt = transform_churn_series(extracted_features_dt = extracted_features_dt, dt_churn = dt['telco_train'])

# significant test and retain only relevent features
relevance_dt = calculate_relevance_table(extracted_features_dt, churn_dt['churn'], ml_task='auto', n_jobs=4)
sig_columns = relevance_dt.loc[relevance_dt['relevant']==True, 'feature'].tolist()
extracted_sig_features_dt = extracted_features_dt[sig_columns]

# extracted settings
tsfresh.feature_extraction.settings.from_columns(extracted_features_dt[sig_columns])

Feature Extraction: 100%|██████████████████████████████████████████████████████████████| 30/30 [00:21<00:00,  1.42it/s]


{'singtel': {'maximum': None,
  'sum_values': None,
  'mean': None,
  'absolute_sum_of_changes': None,
  'linear_trend': [{'attr': 'intercept'}],
  'minimum': None,
  'benford_correlation': None},
 'starhub': {'maximum': None,
  'sum_values': None,
  'mean': None,
  'absolute_sum_of_changes': None,
  'linear_trend': [{'attr': 'intercept'}],
  'benford_correlation': None,
  'minimum': None}}

In [ ]:
extracted_sig_features_dt = extracted_sig_features_dt.reset_index().rename({'index':'msisdn'}, axis=1)
extracted_sig_features_dt.head()

,msisdn,singtel__maximum,starhub__maximum,singtel__sum_values,singtel__mean,starhub__sum_values,starhub__mean,starhub__absolute_sum_of_changes,singtel__absolute_sum_of_changes,"starhub__linear_trend__attr_""intercept""","singtel__linear_trend__attr_""intercept""",singtel__minimum,starhub__benford_correlation,starhub__minimum,singtel__benford_correlation
0,351144701,995.0,805.0,2903.0,483.833333,3221.0,536.833333,1317.0,2307.0,538.619048,477.047619,155.0,-0.210476,245.0,0.250905
1,1041740389,857.0,896.0,2468.0,411.333333,3458.0,576.333333,1697.0,2082.0,403.904762,139.190476,27.0,-0.183830,219.0,0.382712
2,1529121439,1320.0,1038.0,3474.0,579.000000,4280.0,713.333333,1699.0,2270.0,494.761905,350.571429,250.0,-0.074306,44.0,0.442830
3,2189894234,834.0,794.0,3015.0,502.500000,3949.0,658.166667,602.0,1730.0,577.666667,416.857143,129.0,-0.367161,506.0,0.355335
4,2504021639,867.0,755.0,3608.0,601.333333,2812.0,468.666667,1030.0,1055.0,450.809524,555.904762,468.0,-0.002136,226.0,-0.303096


mostly Time series feature that are associated with 
- **singtel**
- **starhub**

In [ ]:
extracted_sig_features_dt.to_pickle((os.path.join("Data","L1", "user_web_TS_feature.pkl")))